In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=0b842c6cb2b7d735a84638ba5b1407cdc8ee5e5e806ea392d028e78560d7e364
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext

# Input

In [97]:
from itertools import permutations
import random
import json

n = 10000

edge_chance = 0.1

node_list = [i + 1 for i in range(n)]

edge_list = []
for edge in permutations(node_list, 2):
    if random.random() < edge_chance:
        edge_list.append(edge)

In [107]:
# node_list = [1, 2, 3, 4]
# edge_list = [(1, 2), (2, 3), (2, 4), (3, 2)]
damping_factor = 0.85

nodes = sc.parallelize(node_list)
node_count = nodes.count()
edges = sc.parallelize(edge_list)

In [67]:
nodes.collect()

[1, 2, 3, 4]

In [68]:
edges.collect()

[(1, 2), (2, 3), (2, 4), (3, 2)]

In [69]:
initial_nodes = nodes.map(lambda node: (node, 1/node_count))

In [70]:
initial_nodes.collect()

[(1, 0.25), (2, 0.25), (3, 0.25), (4, 0.25)]

In [71]:
edge_count = edges.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y).map(lambda x: (x[0], 1/x[1]))

In [72]:
edge_count.collect()

[(2, 0.5), (1, 1.0), (3, 1.0)]

In [73]:
messages = initial_nodes.join(edge_count).map(lambda x: (x[0], x[1][0] * x[1][1]))

In [74]:
messages.collect()

[(1, 0.25), (2, 0.125), (3, 0.25)]

In [66]:
exchange = messages.join(edges)
exchange.collect()

[(1, (0.25, 2)), (2, (0.125, 3)), (2, (0.125, 4)), (3, (0.25, 2))]

In [79]:
exchange_reduce = exchange.map(lambda x: (x[1][1], x[1][0])).reduceByKey(lambda x, y: x + y)
exchange_reduce.collect()

[(2, 0.5), (3, 0.125), (4, 0.125)]

In [77]:
total_page_rank = exchange_reduce.map(lambda x: x[1]).reduce(lambda x, y : x + y)

In [78]:
exchange_reduce = exchange_reduce.map(lambda x: (x[0], x[1] / total_page_rank))
exchange_reduce.collect()

[(2, 0.6666666666666666), (3, 0.16666666666666666), (4, 0.16666666666666666)]

In [30]:
def compute_final_page_rank(x):
  if(x[1][1] is None):
    return (x[0], (1 - damping_factor)/node_count)
  return (x[0], x[1][1]*damping_factor + (1 - damping_factor)/node_count)

In [59]:
final_page_rank = initial_nodes.leftOuterJoin(exchange_reduce).map(compute_final_page_rank)
final_page_rank.collect()

[(1, 0.037500000000000006),
 (2, 0.6041666666666666),
 (3, 0.17916666666666667),
 (4, 0.17916666666666667)]

In [43]:
!pwd

/content


In [108]:
import time
sc.setCheckpointDir("/content/checkpoints")
def computePageRank(nodes, edges, iter):

  node_count = nodes.count()

  initial_nodes = nodes.map(lambda node: (node, 1/node_count)).cache()
  edge_count = edges.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y).map(lambda x: (x[0], 1/x[1])).cache()
  nodes = initial_nodes

  for i in range(iter):
    print(i)
    messages = nodes.join(edge_count, 8).map(lambda x: (x[0], x[1][0] * x[1][1]))

    exchange = messages.join(edges, 8)

    exchange_reduce = exchange.map(lambda x: (x[1][1], x[1][0])).reduceByKey(lambda x, y: x + y).cache()
    total_page_rank = exchange_reduce.map(lambda x: x[1]).reduce(lambda x, y : x + y)
    exchange_reduce = exchange_reduce.map(lambda x: (x[0], x[1] / total_page_rank))

    nodes = initial_nodes.leftOuterJoin(exchange_reduce, 8).map(compute_final_page_rank).cache()

  return nodes.collect()

page_rank = computePageRank(nodes, edges, 3)

0
1
2
